In [ ]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 2. 파일 업로드
from google.colab import files
import json

uploaded = files.upload()

In [ ]:
# 3. 필요한 패키지 설치
!pip install --upgrade transformers datasets accelerate fsspec==2025.3.2 --quiet

In [ ]:
# 4. 주요 라이브러리 로딩
import json
import torch
import wandb
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, EarlyStoppingCallback

In [ ]:
# 5. 모델 및 토크나이저 로딩
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

tokenizer.add_special_tokens({'eos_token': '<END>', 'pad_token': '<pad>'})
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# 6. Train Set 로딩
with open("augmented_knowledge.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

In [ ]:
# 7. Validation Set 로딩
with open("knowledge.json", "r", encoding="utf-8") as f:
    val_data = json.load(f)

In [ ]:
# 8. 문제생성용 프롬프트
def format_data(data):
    return [
        {
            "Text": f"난이도: {item['Level']} \n문제: {item['Question']}<END>",
            "Mask": f"난이도: {item['Level']} \n문제:"
        }
        for item in data
    ]

formatted_train_data = format_data(train_data)
formatted_val_data = format_data(val_data)

In [ ]:
# 9. 마스킹 토크나이즈 함수
def tokenize_with_mask(example):
    full_input = tokenizer(example["Text"], padding="max_length", truncation=True, max_length=512)
    q_ids = tokenizer(example["Mask"], truncation=True, max_length=512)["input_ids"]
    q_len = len(q_ids)

    labels = full_input["input_ids"][:]
    labels[:q_len] = [-100] * q_len
    full_input["labels"] = labels

    return full_input

In [ ]:
# 10. Dataset 변환 및 전처리
train_dataset = Dataset.from_list(formatted_train_data)
val_dataset = Dataset.from_list(formatted_val_data)

tokenized_train_dataset = train_dataset.map(tokenize_with_mask, batched=False, remove_columns=["Text", "Mask"])
tokenized_val_dataset = val_dataset.map(tokenize_with_mask, batched=False, remove_columns=["Text", "Mask"])

In [ ]:
# 11. Trainer 준비
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/KoGPT/CheckPoint/Knowledge/Problem",
    num_train_epochs=20,
    per_device_train_batch_size=2,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=1,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="wandb",
    fp16=True,
    overwrite_output_dir=True,
    learning_rate=1e-5,
    warmup_steps=50
)

In [ ]:
# 12. Trainer 구성
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
# 13. 학습 실행
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: circlehalf17 (circlehalf17-no-job) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
50,3.110200,3.034210
100,2.603300,2.471348
150,1.994800,2.161007
200,2.089100,1.959037
250,1.596300,1.723510
300,1.568000,1.630529
350,1.132300,1.501563
400,1.070000,1.424156
450,0.897600,1.356566
500,0.897900,1.296964


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=1050, training_loss=1.150660584313529, metrics={'train_runtime': 563.1676, 'train_samples_per_second': 7.103, 'train_steps_per_second': 3.551, 'total_flos': 548713267200000.0, 'train_loss': 1.150660584313529, 'epoch': 10.5})

In [ ]:
# 14. 모델 저장
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Knowledge/Problem")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Knowledge/Problem")